In [55]:
import pandas as pd
import numpy as np
import openai
import os
import re
import sys
from langchain.vectorstores import Chroma
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import tiktoken


In [ ]:
# Initialize OpenAI API
openai.api_key = 'add-your-key'

In [ ]:
df = pd.read_csv('/Data/explanations_only_gender.csv') ##gender
#df = pd.read_csv('/Data/explanations_only_age.csv') ##age

In [59]:
# Using tiktoken for GPT-4 tokenizer
tokenizer = tiktoken.encoding_for_model("gpt-4o")


In [ ]:
# Initialize an empty list to store the assistant's replies
assistant_replies = []

# Function to make API requests and handle rate limits
def make_api_request(prompt):
    try:
        response = openai.chat.completions.create(
           
            model = "gpt-4o",
            messages=[
                        {
                            "role": "user",
                            "content": [
                                {
                                "type": "text",
                                "text": prompt
                                }
                            ]
                        },
                      ],
        )
        return response.choices[0].message.content
    except openai.RateLimitError as e:
        # Handle rate limit exceeded error by waiting for a fixed time (e.g., 60 seconds)
        print("Rate limit exceeded. Waiting for 60 seconds...")
        time.sleep(60)
        return make_api_request(prompt)  # Retry the request after waiting


In [61]:
def remove_links(text):
    # Define a regular expression pattern to match URLs
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Remove URLs from the text
    text_without_links = re.sub(url_pattern, '', text)

    return text_without_links

def remove_line_breaks_and_links(text):
    # Remove line breaks
    text_without_line_breaks = re.sub(r'\n{2,}', '\n', text)
    removew_leading_indent = re.sub(r'\n\s+', '\n', text_without_line_breaks)  # Remove leading indentation
    
    # Remove links
    text_without_links = remove_links(removew_leading_indent)

    return text_without_links

In [ ]:
# Iterate through the DataFrame and send each text as a user message
for index, row in df.iterrows():
    text = row['explanation']
    #cleaned_text = remove_line_breaks_and_links(text).strip()
    #gender {uncomment for gender}
    user_message = "Select one theme out of 2 themes from the following lists and assign it to the given text. Don't generate anything else other than selecting just one theme and assign it.\n Themes: (1) Perceived Interests and Roles; (2) Roles as Caregivers, Environmental Advocates, and Socially Conscious Individuals. \n Text:"+text+"\n"
    # age group {uncomment for age}
    #user_message = "Select one theme out of 4 themes from the following lists and assign it to the given text. Don't generate anything else other than selecting just one theme and assign it.\n Themes: Activism and Environmental Consciousness, Proactive and Responsible Mindset, Responsibilities and Concerns, Health and Safety Concerns. \n Text:"+text+"\n"
    token_count = len(tokenizer.encode(user_message))
    #print("Number of tokens:", token_count)
    if (token_count+40) < 4000:
        # Call the function to make the API request and handle rate limits
        assistant_reply = make_api_request(user_message)
        # Append the assistant's reply to the list
        assistant_replies.append(assistant_reply)
    elif (token_count+40) >= 4000:
        # Call the function to make the API request and handle rate limits
        assistant_reply = make_api_request(user_message[:3950])
        # Append the assistant's reply to the list
        assistant_replies.append(assistant_reply)

In [ ]:
# Add the assistant's replies to the DataFrame
df['assistant_reply'] = assistant_replies

In [ ]:
df.to_csv("/Data/LLM_assignment_explanations_only_gender.csv", index=False) #gender
#df.to_csv("/Data/LLM_assignment_explanations_only_age.csv", index=False) #age
